In [1]:
import pandas as pd
import numpy as np
from darts.models import LinearRegressionModel, RandomForest
from pre_processing_model import PreProcessing
from forecast_model import ModelForecast
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
from darts.metrics import mae, mse, rmse
import joblib
%load_ext autoreload

start_date = '2025-03-01'
end_date = '2025-03-15'
# load data from energinet api (default resolution 1 min) and convert to Timeseries
preprocess_data = PreProcessing(start_date, end_date)
preprocess_data.fetch_data()
afrr_energy_UP_ts, afrr_energy_DOWN_ts, past_features_UP_ts, past_features_DOWN_ts, future_features_UP_ts, future_features_DOWN_ts  = preprocess_data.df_to_ts()
output_chunk_length = 30 #minutes


c:\Users\manis\anaconda3\envs\opt_env\Lib\site-packages\statsforecast\utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


Energy UP (Positive) Activation prices forecast model 

In [17]:

model = 'UP'

# define a LR Regressor model
# output_chunk_length = 48
# lr_model = LinearRegressionModel(
#     output_chunk_length=  output_chunk_length,
#     lags = list(range(-1, -120, -5)),
#     lags_past_covariates = list(range(-1, -20, -5)),
#     lags_future_covariates = list(range(-1, -20, -5))
# )

rf_model = RandomForest(
    # lags = list(range(-60, -121, -60)),    #optimal 179
    lags = list(range(-1, -133, -1)),    #optimal 179
    lags_past_covariates = list(range(-1, -11, -1)),  
    lags_future_covariates = list(range(-1, -13, -1)),
    output_chunk_length = output_chunk_length,
    n_estimators = 50,
    max_depth = 12
    # criterion="absolute_error",
)

# use the forecast model to backtest on the timeseries data
forecast = ModelForecast(
    model=rf_model,
    split_size=0.5,
    forecast_horizon = 30, #minutes step
    stride = 30, #minutes
    target=afrr_energy_UP_ts,
    past_covar= past_features_UP_ts,
    future_covar=future_features_UP_ts
)

output_df, output_ts = forecast.backtest_historical_forecast()


`start` position `1` corresponding to time `2025-02-28 23:01:00` is before the first predictable/trainable historical forecasting point for series at index: 0. Using the first historical forecasting point `2025-03-01 01:31:00` that lies a round-multiple of `stride=30` ahead of `start`. To hide these warnings, set `show_warnings=False`.


historical forecasts:   0%|          | 0/666 [00:00<?, ?it/s]

<function mae at 0x000001D6F4F26020> on the test set: 0.03
<function mse at 0x000001D6F4F267A0> on the test set: 0.00
<function rmse at 0x000001D6F4F26D40> on the test set: 0.05


In [18]:

# re-normalise the output dataset
if model == 'UP':
    scaler = joblib.load('scaler_UP.pkl')
    actual_ts = scaler.inverse_transform(afrr_energy_UP_ts)
    forecasted_ts = scaler.inverse_transform(output_ts)
elif model == 'DOWN':
    scaler = joblib.load('scaler_DOWN.pkl')
    actual_ts = scaler.inverse_transform(afrr_energy_DOWN_ts)
    forecasted_ts = scaler.inverse_transform(output_ts)

actual_ts_train, actual_ts_test = actual_ts.split_after(split_point =0.5)
forecasted_ts_train, forecasted_ts_test = forecasted_ts.split_after(split_point =0.5)

Y_df = pd.merge(actual_ts_test.pd_dataframe(), forecasted_ts_test.pd_dataframe(), left_index=True, right_index=True, how='right')
Y_df = Y_df.rename(columns= {'aFRR_UpActivatedPriceEUR_x': 'aFRR_UpActivatedPriceEUR', 'aFRR_UpActivatedPriceEUR_y': 'aFRR_UP_Price_prediction'})

# error metrics
print(f"MAE_ts: {mae(actual_ts_test, forecasted_ts_test):.2f}")
print(f"MSE_ts: {mse(actual_ts_test, forecasted_ts_test):.2f}")
print(f"RMSE_ts: {rmse(actual_ts_test, forecasted_ts_test):.2f}")

# plot the results
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=Y_df.index, y=Y_df['aFRR_UP_Price_prediction'], mode='lines', name='Forecasted Price Up'))
fig.add_trace(go.Scatter(x=Y_df.index, y=Y_df['aFRR_UpActivatedPriceEUR'], mode='lines', name='Actual Price UP'))
fig.update_layout(title=f'Forecast vs Actual price {model}',xaxis_title='Time',yaxis_title='Price (EUR/MWh)',legend_title='Legend',template='plotly_white')
fig.show()

MAE_ts: 69.82
MSE_ts: 17627.87
RMSE_ts: 132.77


In [19]:
# plot trend of price over a day - averaged per minute

Y_df['Minuteoftheday'] =Y_df.index.minute  + Y_df.index.hour * 60

daily_price_trend_on_min_avg = Y_df.groupby('Minuteoftheday')[['aFRR_UpActivatedPriceEUR', 'aFRR_UP_Price_prediction']].mean().reset_index()
daily_price_trend_on_min_avg['Hour'] = pd.to_datetime(daily_price_trend_on_min_avg['Minuteoftheday'], unit='m').dt.strftime('%H:%M')
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_price_trend_on_min_avg['Hour'], y=daily_price_trend_on_min_avg['aFRR_UP_Price_prediction'], mode='lines', name='Forecasted Price Up'))
fig.add_trace(go.Scatter(x=daily_price_trend_on_min_avg['Hour'], y=daily_price_trend_on_min_avg['aFRR_UpActivatedPriceEUR'], mode='lines', name='Actual Price UP'))
fig.update_layout(title=f'Forecast vs Actual price {model} over a day',xaxis_title='Time',yaxis_title='Price (EUR/MWh)',legend_title='Legend',template='plotly_white')
fig.show()

Energy DOWN (Negative) Activation prices model

In [14]:
model = 'DOWN'

# define a LR Regressor model
# output_chunk_length = 48
# lr_model = LinearRegressionModel(
#     output_chunk_length=  output_chunk_length,
#     lags = list(range(-1, -120, -5)),
#     lags_past_covariates = list(range(-1, -20, -5)),
#     lags_future_covariates = list(range(-1, -20, -5))
# )

rf_model = RandomForest(
    # lags = list(range(-60, -121, -60)),    #optimal 179
    lags = list(range(-1, -120, -1)),    #optimal 179
    lags_past_covariates = list(range(-1, -10, -1)),  
    lags_future_covariates = list(range(-1, -10, -1)),
    output_chunk_length = output_chunk_length,
    n_estimators = 50,
    max_depth = 10
    # criterion="absolute_error",
)

# use the forecast model to backtest on the timeseries data
forecast = ModelForecast(
    model=rf_model,
    split_size=0.5,
    forecast_horizon = 30, #minutes step
    stride = 30, #minutes
    target=afrr_energy_DOWN_ts,
    past_covar= past_features_DOWN_ts,
    future_covar=future_features_DOWN_ts
)

output_df, output_ts = forecast.backtest_historical_forecast()


`start` position `1` corresponding to time `2025-02-28 23:01:00` is before the first predictable/trainable historical forecasting point for series at index: 0. Using the first historical forecasting point `2025-03-01 01:01:00` that lies a round-multiple of `stride=30` ahead of `start`. To hide these warnings, set `show_warnings=False`.


historical forecasts:   0%|          | 0/667 [00:00<?, ?it/s]

<function mae at 0x000001D6F4F26020> on the test set: 0.00
<function mse at 0x000001D6F4F267A0> on the test set: 0.00
<function rmse at 0x000001D6F4F26D40> on the test set: 0.02


In [16]:

# re-normalise the output dataset
if model == 'UP':
    scaler = joblib.load('scaler_UP.pkl')
    actual_ts = scaler.inverse_transform(afrr_energy_UP_ts)
    forecasted_ts = scaler.inverse_transform(output_ts)
elif model == 'DOWN':
    scaler = joblib.load('scaler_DOWN.pkl')
    actual_ts = scaler.inverse_transform(afrr_energy_DOWN_ts)
    forecasted_ts = scaler.inverse_transform(output_ts)

actual_ts_train, actual_ts_test = actual_ts.split_after(split_point =0.5)
forecasted_ts_train, forecasted_ts_test = forecasted_ts.split_after(split_point =0.5)

Y_df = pd.merge(actual_ts_test.pd_dataframe(), forecasted_ts_test.pd_dataframe(), left_index=True, right_index=True, how='right')
Y_df = Y_df.rename(columns= {'aFRR_DownActivatedPriceEUR_x': 'aFRR_DownActivatedPriceEUR', 'aFRR_DownActivatedPriceEUR_y': 'aFRR_DOWN_Price_prediction'})

# error metrics
print(f"MAE_ts: {mae(actual_ts_test, forecasted_ts_test):.2f}")
print(f"MSE_ts: {mse(actual_ts_test, forecasted_ts_test):.2f}")
print(f"RMSE_ts: {rmse(actual_ts_test, forecasted_ts_test):.2f}")

# plot the results
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=Y_df.index, y=Y_df['aFRR_DOWN_Price_prediction'], mode='lines', name='Forecasted Price Up'))
fig.add_trace(go.Scatter(x=Y_df.index, y=Y_df['aFRR_DownActivatedPriceEUR'], mode='lines', name='Actual Price UP'))
fig.update_layout(title=f'Forecast vs Actual price {model}',xaxis_title='Time',yaxis_title='Price (EUR/MWh)',legend_title='Legend',template='plotly_white')
fig.show()

MAE_ts: 23.24
MSE_ts: 24365.84
RMSE_ts: 156.10
